# 1. DBBE

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics import adjusted_rand_score, v_measure_score
from tqdm import tqdm
from collections import defaultdict
import igraph as ig
import leidenalg as la
import faiss
import os
from typing import Dict
import re
import unicodedata
import pandas as pd
from collections import defaultdict
from itertools import combinations

os.makedirs('dbbe_semantic_results', exist_ok=True)

csv_path = 'dbbe_full.csv'
df = pd.read_csv(csv_path)
df = df.dropna()

model_name = 'kevinkrahn/shlm-grc-en'
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.eval()

def cls_pooling(model_output):
    return model_output[0][:, 0]

verses = df['verse'].tolist()
embeddings = []
batch_size = 32

for i in tqdm(range(0, len(verses), batch_size), desc="Embeddings"):
    batch = verses[i:i+batch_size]
    encoded_input = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    
    with torch.no_grad():
        model_output = model(**encoded_input)
    
    batch_embeddings = cls_pooling(model_output).cpu().numpy()
    embeddings.append(batch_embeddings)

embeddings = np.vstack(embeddings)
embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

dimension = embeddings.shape[1]
nlist = 500
k = 50
nprobe = 10

quantizer = faiss.IndexFlatIP(dimension)
index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_INNER_PRODUCT)

index.train(embeddings.astype('float32'))
index.add(embeddings.astype('float32'))
index.nprobe = nprobe
distances, indices = index.search(embeddings.astype('float32'), k)

similarity_thresholds = [0.70, 0.75, 0.80, 0.85, 0.90]
threshold_results = []
all_resolution_results = []

for sim_threshold in similarity_thresholds:
    print(f"\n{'='*80}")
    print(f"Threshold: {sim_threshold:.2f}")
    print(f"{'='*80}")
    
    candidate_pairs = set()
    
    for i in tqdm(range(len(embeddings)), desc=f"Building graph (t={sim_threshold:.2f})"):
        for j_idx, distance in zip(indices[i], distances[i]):
            if j_idx != i and j_idx != -1:
                similarity = distance
                if similarity >= sim_threshold:
                    idx1, idx2 = min(i, j_idx), max(i, j_idx)
                    candidate_pairs.add((idx1, idx2))
    
    n_pairs = len(candidate_pairs)
    avg_degree = n_pairs * 2 / len(embeddings)
    
    print(f"Candidate pairs: {n_pairs:,} (avg degree: {avg_degree:.1f})")
    
    if n_pairs == 0:
        print("No pairs found - skipping")
        threshold_results.append({
            'threshold': sim_threshold,
            'n_pairs': 0,
            'best_resolution': None,
            'best_ari': 0,
            'best_v_measure': 0,
            'n_clusters': 0
        })
        continue
    
    edges = []
    weights = []
    
    for i, j in tqdm(candidate_pairs, desc="Edge weights"):
        sim = float(np.dot(embeddings[i], embeddings[j]))
        edges.append((i, j))
        weights.append(sim)
    
    g = ig.Graph(n=len(embeddings), edges=edges, directed=False)
    g.es['weight'] = weights
    
    print(f"Graph: {g.vcount()} nodes, {g.ecount()} edges")
    
    w = np.array(weights)
    w_scaled = ((w - w.min()) / (w.max() - w.min())) ** 3
    g.es['weight'] = w_scaled.tolist()
    
    hub_thresh = 500
    for v in range(g.vcount()):
        if g.degree(v) > hub_thresh:
            for e in g.incident(v):
                g.es[e]['weight'] *= 0.5
    
    print("Leiden resolution sweep...")
    resolutions = np.logspace(-2, 1, 20)
    
    best_ari = -1
    best_labels = None
    best_res = None
    best_v = None
    
    for res in tqdm(resolutions, desc="Resolutions"):
        partition = la.find_partition(
            g,
            la.CPMVertexPartition,
            weights='weight',
            resolution_parameter=res,
            n_iterations=-1
        )
        labels = np.array(partition.membership)
        ari = adjusted_rand_score(df['idgroup'], labels)
        v_measure = v_measure_score(df['idgroup'], labels)
        n_clusters = len(set(labels))
        
        col_name = f'cluster_t{int(sim_threshold*100)}_r{res:.6f}'
        df[col_name] = labels
        
        all_resolution_results.append({
            'threshold': sim_threshold,
            'resolution': res,
            'ari': ari,
            'v_measure': v_measure,
            'n_clusters': n_clusters,
            'column_name': col_name
        })
        
        if ari > best_ari:
            best_ari = ari
            best_labels = labels
            best_res = res
            best_v = v_measure
    
    n_clusters = len(set(best_labels))
    
    print(f"Best resolution: {best_res:.4f}")
    print(f"Best ARI: {best_ari:.4f}")
    print(f"Best V-measure: {best_v:.4f}")
    print(f"Clusters: {n_clusters}")
    
    threshold_results.append({
        'threshold': sim_threshold,
        'n_pairs': n_pairs,
        'avg_degree': avg_degree,
        'best_resolution': best_res,
        'best_ari': best_ari,
        'best_v_measure': best_v,
        'n_clusters': n_clusters
    })
    
    df[f'cluster_t{int(sim_threshold*100)}_best'] = best_labels

print("\n" + "="*80)
print("GRID SEARCH RESULTS (BEST PER THRESHOLD)")
print("="*80)

results_df = pd.DataFrame(threshold_results)
print(results_df.to_string(index=False))

best_threshold_row = results_df.loc[results_df['best_ari'].idxmax()]
best_threshold = best_threshold_row['threshold']

print(f"\n{'='*80}")
print(f"OVERALL BEST THRESHOLD: {best_threshold:.2f}")
print(f"{'='*80}")
print(f"ARI: {best_threshold_row['best_ari']:.4f}")
print(f"V-measure: {best_threshold_row['best_v_measure']:.4f}")
print(f"Resolution: {best_threshold_row['best_resolution']:.4f}")
print(f"Clusters: {int(best_threshold_row['n_clusters'])}")

df['cluster_best'] = df[f'cluster_t{int(best_threshold*100)}_best']

df.to_csv("dbbe_semantic_results/faiss_leiden_gridsearch_results.csv", index=False)
results_df.to_csv("dbbe_semantic_results/threshold_gridsearch_summary.csv", index=False)

all_resolution_df = pd.DataFrame(all_resolution_results)
all_resolution_df.to_csv("dbbe_semantic_results/all_threshold_resolution_results.csv", index=False)

print("\nFiles saved:")
print("  - dbbe_semantic_results/faiss_leiden_gridsearch_results.csv")
print("  - dbbe_semantic_results/threshold_gridsearch_summary.csv")
print("  - dbbe_semantic_results/all_threshold_resolution_results.csv")

print(f"\nTotal clustering columns created: {len(all_resolution_results)}")
print(f"Thresholds tested: {len(similarity_thresholds)}")
print(f"Resolutions per threshold: {len(resolutions)}")

def calculate_jaccard_similarity(clusters_a, clusters_b):
    if not clusters_a or not clusters_b:
        return 0.0
    intersection = len(clusters_a & clusters_b)
    union = len(clusters_a | clusters_b)
    return intersection / union

def reconstruct_poems(df):
    poem_to_clusters = defaultdict(set)
    poem_verse_counts = defaultdict(int)

    for _, row in df.iterrows():
        poem_id = row['idoriginal_poem']
        cluster_id = row['cluster_leiden_fixed']
        poem_verse_counts[poem_id] += 1
        if cluster_id != -1:
            poem_to_clusters[poem_id].add(cluster_id)

    print(f"\nReconstructed {len(poem_to_clusters)} poems")
    return poem_to_clusters, poem_verse_counts

def evaluate_against_ground_truth(df, poem_clusters):
    poem_to_type = df.groupby('idoriginal_poem')['type_id'].first().to_dict()

    y_true = []
    y_pred = []
    for poem_id, predicted_cluster in poem_clusters.items():
        if poem_id in poem_to_type:
            y_true.append(poem_to_type[poem_id])
            y_pred.append(predicted_cluster)

    ari = adjusted_rand_score(y_true, y_pred)
    v_measure = v_measure_score(y_true, y_pred)

    return ari, v_measure, y_true, y_pred

def cluster_poems_jaccard(poem_to_clusters, similarity_threshold=0.66):
    poem_ids = list(poem_to_clusters.keys())
    n_poems = len(poem_ids)

    edges = []
    for i in range(n_poems):
        for j in range(i + 1, n_poems):
            pid_a, pid_b = poem_ids[i], poem_ids[j]
            sim = calculate_jaccard_similarity(poem_to_clusters[pid_a], poem_to_clusters[pid_b])
            if sim >= similarity_threshold:
                edges.append((pid_a, pid_b))

    class UF:
        def __init__(self, elements):
            self.parent = {e: e for e in elements}
            self.rank = {e: 0 for e in elements}

        def find(self, x):
            if self.parent[x] != x:
                self.parent[x] = self.find(self.parent[x])
            return self.parent[x]

        def union(self, x, y):
            px, py = self.find(x), self.find(y)
            if px == py: return
            if self.rank[px] < self.rank[py]: px, py = py, px
            self.parent[py] = px
            if self.rank[px] == self.rank[py]: self.rank[px] += 1

    uf = UF(poem_ids)
    for a, b in edges:
        uf.union(a, b)

    poem_clusters = {pid: uf.find(pid) for pid in poem_ids}
    return poem_clusters, edges

def calculate_perfect_reconstruction_rate(df, poem_clusters):
    poem_to_type = df.groupby('idoriginal_poem')['type_id'].first().to_dict()

    gt_to_poems = defaultdict(set)
    for poem_id, gt_type in poem_to_type.items():
        gt_to_poems[gt_type].add(poem_id)

    pred_to_poems = defaultdict(set)
    for poem_id, pred_cluster in poem_clusters.items():
        pred_to_poems[pred_cluster].add(poem_id)

    perfectly_reconstructed = 0
    total_gt_clusters = len(gt_to_poems)

    for gt_type, gt_poems in gt_to_poems.items():
        for pred_cluster, pred_poems in pred_to_poems.items():
            if gt_poems == pred_poems:
                perfectly_reconstructed += 1
                break

    reconstruction_rate = perfectly_reconstructed / total_gt_clusters if total_gt_clusters > 0 else 0
    return reconstruction_rate, perfectly_reconstructed, total_gt_clusters

df = pd.read_csv("dbbe_semantic_results/faiss_leiden_gridsearch_results.csv")

if 'cluster_best' in df.columns:
    df['cluster_leiden_fixed'] = df['cluster_best']
else:
    raise ValueError("Column 'cluster_best' not found in CSV")

poem_to_clusters, _ = reconstruct_poems(df)

thresholds = [0.50, 0.60, 0.70, 0.8]
results = []

for thresh in thresholds:
    print(f"\nThreshold {thresh:.0%}...")
    poem_clusters, edges = cluster_poems_jaccard(poem_to_clusters, thresh)
    df['poem_cluster_id'] = df['idoriginal_poem'].map(poem_clusters)

    ari, v_measure, _, _ = evaluate_against_ground_truth(df, poem_clusters)
    reconstruction_rate, n_perfect, n_total_gt = calculate_perfect_reconstruction_rate(df, poem_clusters)

    results.append({
        'threshold': thresh,
        'n_poem_clusters': len(set(poem_clusters.values())),
        'n_edges': len(edges),
        'ari': ari,
        'v_measure': v_measure,
        'perfect_reconstruction_rate': reconstruction_rate,
        'n_perfect_clusters': n_perfect,
        'n_total_gt_clusters': n_total_gt
    })

results_df = pd.DataFrame(results)
df.to_csv('dbbe_semantic_results/dbbe_poems_semantic.csv')
results_df.to_csv('dbbe_semantic_results/dbbe_poems_semantic_stats.csv')

results_df = results_df.sort_values('threshold')

plt.figure(figsize=(6,4))
plt.plot(results_df['threshold'], results_df['ari'], marker='o', linestyle='-')
plt.xticks(results_df['threshold'])
plt.xlabel("Jaccard Similarity Threshold")
plt.ylabel("Adjusted Rand Index (ARI)")
plt.title("ARI of Poem Clustering vs Threshold")
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.savefig("dbbe_semantic_results/ari_poemlevel_sem_dbbe.png", dpi=300)
plt.close()

df = pd.read_csv("dbbe_semantic_results/all_threshold_resolution_results.csv")

df = df[~df['threshold'].isin([0.85, 0.75])]
df = df[df['resolution'] <= 1.0]

df['resolution'] = df['resolution'].round(4)
df['threshold'] = df['threshold'].round(4)
df['ari'] = df['ari'].round(4)

heatmap_data = df.pivot(index='resolution', columns='threshold', values='ari')

plt.figure(figsize=(8, 6))
sns.heatmap(
    heatmap_data, 
    annot=True, 
    fmt=".4f", 
    cmap="viridis", 
    cbar_kws={'label': 'ARI'},
    annot_kws={"fontsize":14}
)
plt.ylabel("Resolution")
plt.xlabel("Similarity Threshold")
plt.title("ARI heatmap across Threshold and Resolution", fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout()

plt.savefig("dbbe_semantic_results/ari_verselevel_sem_dbbe.png", dpi=300)
plt.close()

print("\nAll outputs saved to dbbe_semantic_results/")

Embeddings: 100%|██████████| 1295/1295 [00:40<00:00, 32.28it/s]



Threshold: 0.70


Building graph (t=0.70): 100%|██████████| 41424/41424 [00:04<00:00, 8996.14it/s]


Candidate pairs: 1,608,598 (avg degree: 77.7)


Edge weights: 100%|██████████| 1608598/1608598 [00:03<00:00, 526930.44it/s]


...

Candidate pairs: 652,215 (avg degree: 31.5)


Edge weights: 100%|██████████| 652215/652215 [00:01<00:00, 553975.71it/s]


Graph: 41424 nodes, 652215 edges
Leiden resolution sweep...


Resolutions:  65%|██████▌   | 13/20 [01:49<00:57,  8.22s/it]/tmp/ipykernel_305272/3241684344.py:150: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = labels
Resolutions:  70%|███████   | 14/20 [01:50<00:36,  6.01s/it]/tmp/ipykernel_305272/3241684344.py:150: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col_name] = labels
Resolutions:  75%|███████▌  | 15/20 [01:51<00:22,  4.47s/it]/tmp/ipykernel_305272/3241684344.py:150: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `fra

Best resolution: 0.0100
Best ARI: 0.7847
Best V-measure: 0.9694
Clusters: 12030

GRID SEARCH RESULTS (BEST PER THRESHOLD)
 threshold  n_pairs  avg_degree  best_resolution  best_ari  best_v_measure  n_clusters
      0.70  1608598   77.665025         0.183298  0.655353        0.939715        7081
      0.75  1607928   77.632677         0.183298  0.684695        0.954551        9218
      0.80  1584859   76.518878         0.127427  0.706170        0.959314       10088
      0.85  1350979   65.226873         0.042813  0.723863        0.958053        9869
      0.90   652215   31.489716         0.010000  0.784729        0.969419       12030

OVERALL BEST THRESHOLD: 0.90
ARI: 0.7847
V-measure: 0.9694
Resolution: 0.0100
Clusters: 12030

Files saved:
  - dbbe_semantic_results/faiss_leiden_gridsearch_results.csv
  - dbbe_semantic_results/threshold_gridsearch_summary.csv
  - dbbe_semantic_results/all_threshold_resolution_results.csv

Total clustering columns created: 100
Thresholds tested: 5
Res

In [5]:
# Helper printing verses that are orthographically dissimilar but share a cluster
CLEAN_PATTERN = re.compile(r'[^\w\s]')
WHITESPACE_PATTERN = re.compile(r'\s+')

def preprocess_text(text: str, options: Dict[str, bool] = None) -> str:
    if options is None:
        options = {'lowercase': True, 'remove_diacritics': True, 'remove_punctuation': True}
    text = str(text)
    if options.get('lowercase', True):
        text = text.lower()
    if options.get('remove_diacritics', True):
        text = unicodedata.normalize('NFD', text)
        text = ''.join(char for char in text if unicodedata.category(char) != 'Mn')
        text = unicodedata.normalize('NFC', text)
    else:
        text = unicodedata.normalize('NFC', text)
    if options.get('remove_punctuation', True):
        text = CLEAN_PATTERN.sub('', text)
    text = WHITESPACE_PATTERN.sub(' ', text)
    return text.strip()

df = pd.read_csv("dbbe_semantic_results/dbbe_poems_semantic.csv")
poems = (
    df.sort_values(["idoriginal_poem", "order"])
      .groupby("idoriginal_poem")["verse"]
      .apply(list)
      .to_dict()
)

poem_clusters = (
    df.groupby("idoriginal_poem")["poem_cluster_id"]
      .first()
      .to_dict()
)

cluster_to_poems = defaultdict(list)
for pid, cid in poem_clusters.items():
    cluster_to_poems[cid].append(pid)
def versewise_min_distance(v1, v2):
    v1_proc = [preprocess_text(v) for v in v1]
    v2_proc = [preprocess_text(v) for v in v2]
    
    unmatched = set(range(len(v2_proc)))
    total_dist = 0
    
    for va in v1_proc:
        if not unmatched:
            break
        best_idx = min(unmatched, key=lambda i: jaccard_distance(va, v2_proc[i]))
        total_dist += jaccard_distance(va, v2_proc[best_idx])
        unmatched.remove(best_idx)
        
    return total_dist / max(len(v1_proc), len(v2_proc))

def shingles(s, k=3):
    s = preprocess_text(s)
    return {s[i:i+k] for i in range(len(s)-k+1)} if len(s) >= k else {s}

def jaccard_distance(a, b):

    A, B = shingles(a), shingles(b)
    inter = len(A & B)
    union = len(A | B)
    return 1 - (inter / union if union else 0)

results = [] 

MAX_VERSES = 4
MAX_CLUSTER_SIZE = 5  

for cid, poem_ids in cluster_to_poems.items():
    if len(poem_ids) > MAX_CLUSTER_SIZE:
        continue

    if len(poem_ids) < 2:
        continue

    for p1, p2 in combinations(poem_ids, 2):
        v1, v2 = poems[p1], poems[p2]

        verse_count = len(v1)

        if verse_count > MAX_VERSES:
            continue

        dist = versewise_min_distance(v1, v2)

        results.append((cid, p1, p2, dist, verse_count))

results.sort(key=lambda x: x[3], reverse=True)  

filtered_results = [r for r in results if r[4] > 1]

TOP = 20
for cid, p1, p2, dist, vcount in filtered_results[:TOP]:
    print(f"\n=== Cluster {cid} | Poems in cluster <5 | Verses {vcount} | Distance {dist:.3f} ===")

    print(f"\nPoem {p1}:")
    for line in poems[p1]:
        print("  ", line)

    print(f"\nPoem {p2}:")
    for line in poems[p2]:
        print("  ", line)


=== Cluster 31705 | Poems in cluster <5 | Verses 2 | Distance 0.858 ===

Poem 31679:
   : ἰώνι(ον) μ(ὲν) ἀττ(ι)(κ)(ῶς) μέγα γράφε
   ἄλλ(ως) (δὲ), μικρ(ὸν) οὐχ ἁμαρτήσ(εις) γράφ(ων):-

Poem 31705:
   τ(ὸν) παιονισμ(ὸν) βουλγαρισμ(ὸν) | δ’ εἰ γρ(άφ)(εις), :
   γρ(άφ)(ειν) μι(κ)(ρ)(ὸν) γίνωσκε: | μηδαμῶς μέγα:-

=== Cluster 36240 | Poems in cluster <5 | Verses 2 | Distance 0.741 ===

Poem 36240:
   ⁘ ὁ τῆς ἐνεδρ(ας) τω(ν) πονηρ(ων) δαιμο(νων)
   ῥυσθεις ἀνυμνεῖ τὸν λυτρωτ(ην) ἐκ πόθου ⁘

Poem 36419:
   ⁘ ἐχθρῶν πονηρω(ν) δαιμο(νων) τυραννι(...)
   αιτει λυτρω(...) (...) ⁘

=== Cluster 17726 | Poems in cluster <5 | Verses 3 | Distance 0.643 ===

Poem 17726:
   + ὧσπερ ξένη χέροντες, ἡδῆν π(ατ)ρίδα·
   καὶ η θαλατ(τέ)βοντες του ευρῆν λημέ(νας?).
   οῦτος και ἡ γρά|φοντες εὔρην βηβλῆου τελος.

Poem 18561:
   + ὧσπερ ξένοι χάιρουσιν εὑρεῖν π(ατ)ρῖδα.
   καὶ οἱ θαλαττεύοντες εὑρεῖν λημ(έν)α.
   οὗτω καὶ οἰ γράφοντες βλϊβλίου τέλος.

=== Cluster 17996 | Poems in cluster <5 | Verses 3 | Distan

In [66]:
# Helper printing all verses that were clustered together with a given target idgroup
df = pd.read_csv("dbbe_semantic_results/dbbe_poems_semantic.csv")

target_idgroup = 831
threshold_cols = ['cluster_t70', 'cluster_t75', 'cluster_t80', 'cluster_t85', 'cluster_t90', 'cluster_leiden_fixed']
poem_to_cluster = df.groupby('idoriginal_poem')['poem_cluster_id'].first().to_dict()


verse_row = df[df['idgroup'] == target_idgroup]
if verse_row.empty:
    print("idgroup not found")
else:
    print(f"Verse {target_idgroup}: {verse_row['verse'].iloc[0]}\n")dbbe_semantic_results

    for col in threshold_cols:
        cluster_id = verse_row[col].iloc[0]
        same_cluster = df[df[col] == cluster_id]
        same_cluster_sorted = same_cluster.sort_values(['idoriginal_poem', 'order'])

        poem_ids_in_cluster = same_cluster_sorted['idoriginal_poem'].unique().tolist()

        print(f"=== {col} | Cluster {cluster_id} | {len(poem_ids_in_cluster)} poems ===")

        for _, row in same_cluster_sorted.iterrows():
            poem_id = row['idoriginal_poem']
            verse_text = row['verse']
            type_id = row['type_id']
            verse_group = row['idgroup']

            poem_cluster = poem_to_cluster.get(poem_id, "N/A")
            print(f"  Poem ID: {poem_id} | Verse: {verse_text} | Poem Cluster: {poem_cluster} | Type ID: {type_id} | Verse Group {verse_group}")

        print("\n")


Verse 19026: ποντωπλο|οῦσι παῦλα λιμὴν | τῶν πόνων·​

=== cluster_t70 | Cluster 2427 | 4 poems ===
  Poem ID: 18986 | Verse: ποντωπλο|οῦσι παῦλα λιμὴν | τῶν πόνων·​ | Poem Cluster: 18986 | Type ID: 3096 | Verse Group 19026.0
  Poem ID: 21229 | Verse: ναυκρατίου μεγάλου γλυ|κερὸς πόνος ὅν ποτε κεῖνος: |  | Poem Cluster: 17481 | Type ID: 2019 | Verse Group 19072.0
  Poem ID: 22021 | Verse: Ποντοπλωοῦσι παῦλα· λιμὶν τῶν πόνων· | Poem Cluster: 22021 | Type ID: 3096 | Verse Group 19026.0
  Poem ID: 35833 | Verse: ποντοποοῦσι παῦλα· λυμὴν τῶ(ν) πόνων· | Poem Cluster: 18986 | Type ID: 3096 | Verse Group 19026.0


=== cluster_t75 | Cluster 3465 | 3 poems ===
  Poem ID: 18986 | Verse: ποντωπλο|οῦσι παῦλα λιμὴν | τῶν πόνων·​ | Poem Cluster: 18986 | Type ID: 3096 | Verse Group 19026.0
  Poem ID: 22021 | Verse: Ποντοπλωοῦσι παῦλα· λιμὶν τῶν πόνων· | Poem Cluster: 22021 | Type ID: 3096 | Verse Group 19026.0
  Poem ID: 35833 | Verse: ποντοποοῦσι παῦλα· λυμὴν τῶ(ν) πόνων· | Poem Cluster: 18986 | Type

In [76]:
# Helper printing all poems for a poem_cluster_id
df = pd.read_csv("dbbe_semantic_results/dbbe_poems_semantic.csv")
target_cluster_id = 36240

cluster_df = df[df['poem_cluster_id'] == target_cluster_id]

poems_in_cluster = (
    cluster_df.sort_values(['idoriginal_poem', 'order'])
              .groupby('idoriginal_poem')['verse']
              .apply(list)
              .to_dict()
)

for poem_id, verses in poems_in_cluster.items():
    print(f"\nPoem {poem_id}:")
    for line in verses:
        print("  ", line)


Poem 36240:
   ⁘ ὁ τῆς ἐνεδρ(ας) τω(ν) πονηρ(ων) δαιμο(νων)
   ῥυσθεις ἀνυμνεῖ τὸν λυτρωτ(ην) ἐκ πόθου ⁘

Poem 36419:
   ⁘ ἐχθρῶν πονηρω(ν) δαιμο(νων) τυραννι(...)
   αιτει λυτρω(...) (...) ⁘


# 2. Full dataset

In [2]:
# import re
# import unicodedata
# from typing import Dict
# import pandas as pd
# import numpy as np
# from transformers import AutoTokenizer, AutoModel
# import torch
# from tqdm import tqdm
# from collections import defaultdict, Counter
# import time
# import os
# import pickle
# import gzip
# from pathlib import Path

# CHECKPOINT_DIR = Path("/scratch/gent/vo/000/gvo00042/vsc48660/full_semantic_clustering_checkpoints")
# CHECKPOINT_DIR.mkdir(exist_ok=True)

# print("✓ Imports loaded")
# print(f"✓ Checkpoint directory: {CHECKPOINT_DIR}")

# CLEAN_PATTERN = re.compile(r'[^\w\s]')
# WHITESPACE_PATTERN = re.compile(r'\s+')

# def preprocess_text(text: str, options: Dict[str, bool] = None) -> str:
#     if options is None:
#         options = {'lowercase': True, 'remove_diacritics': True, 'remove_punctuation': True}
#     text = str(text)
#     if options.get('lowercase', True):
#         text = text.lower()
#     if options.get('remove_diacritics', True):
#         text = unicodedata.normalize('NFD', text)
#         text = ''.join(char for char in text if unicodedata.category(char) != 'Mn')
#         text = unicodedata.normalize('NFC', text)
#     else:
#         text = unicodedata.normalize('NFC', text)
#     if options.get('remove_punctuation', True):
#         text = CLEAN_PATTERN.sub('', text)
#     text = WHITESPACE_PATTERN.sub(' ', text)
#     return text.strip()

# def cls_pooling(model_output):
#     return model_output[0][:, 0]

# print("✓ Utility functions defined")

# embeddings_file = CHECKPOINT_DIR / 'embeddings.npz'
# metadata_file = CHECKPOINT_DIR / 'metadata.pkl.gz'

# if embeddings_file.exists() and metadata_file.exists():
#     print("="*80)
#     print("📂 COMPACT CHECKPOINT FOUND - LOADING FROM DISK")
#     print("="*80)
    
#     print("\nLoading embeddings...")
#     embeddings_data = np.load(embeddings_file)
#     embeddings = embeddings_data['embeddings']
#     print(f"  ✓ Embeddings: {embeddings.shape}")
    
#     pairs_file = CHECKPOINT_DIR / 'candidate_pairs.npz'
#     if pairs_file.exists():
#         print("Loading candidate pairs...")
#         pairs_data = np.load(pairs_file)
#         pairs_array = pairs_data['pairs']
#         candidate_pairs = set((int(i), int(j)) for i, j in pairs_array)
#         print(f"  ✓ Candidate pairs: {len(candidate_pairs):,}")
#     else:
#         candidate_pairs = set()
    
#     print("Loading DataFrame...")
#     df = pd.read_parquet(CHECKPOINT_DIR / 'df_minimal.parquet')
#     print(f"  ✓ DataFrame: {len(df):,} rows")
    
#     print("Loading metadata...")
#     with gzip.open(metadata_file, 'rb') as f:
#         metadata = pickle.load(f)
    
#     source_datasets = metadata['source_datasets']
#     dataset_to_indices = metadata['dataset_to_indices']
#     similarity_threshold = metadata['similarity_threshold']
#     faiss_config = metadata['faiss_config']
    
#     dimension = faiss_config['dimension']
#     nlist = faiss_config['nlist']
#     k = faiss_config['k']
#     nprobe = faiss_config['nprobe']
    
#     total_size = sum(os.path.getsize(f) / 1024 / 1024 
#                      for f in CHECKPOINT_DIR.glob('*') 
#                      if f.is_file() and f.suffix in ['.npz', '.parquet', '.gz'])
    
#     print(f"\n✅ Checkpoint loaded! Size: {total_size:.1f} MB")
#     print(f"  Documents: {len(df):,}")
#     print(f"  Candidate pairs: {len(candidate_pairs):,}")
#     print(f"  Source datasets: {list(dataset_to_indices.keys())}")
    
#     print("\n" + "="*80)
#     print("✅ READY TO PROCEED WITH LEIDEN STAGE")
#     print("="*80)
    
#     LOADED_FROM_CHECKPOINT = True
#     faiss_available = True
    
# else:
#     print("="*80)
#     print("ℹ️  NO CHECKPOINT FOUND")
#     print("="*80)
#     print("Will run pipeline from the beginning.\n")
#     LOADED_FROM_CHECKPOINT = False


# if not LOADED_FROM_CHECKPOINT:
#     print("="*80)
#     print("LOADING DATA")
#     print("="*80)
    
#     csv_path = 'concatenated.csv'
#     df = pd.read_csv(csv_path)
#     df = df.dropna(subset=['verse', 'source_dataset'])
#     df = df[df['verse'].fillna('').astype(str).str.len() >= 20]
#     df = df[df['verse'].fillna('').astype(str).str.len() < 256]
#     df['verse'] = df['verse'].apply(preprocess_text)
#     df = df.reset_index(drop=True)
    
#     print(f"Total verses: {len(df):,}")
#     print(f"Dataset distribution:")
#     print(df['source_dataset'].value_counts())
    
#     # Load model
#     model_name = 'kevinkrahn/shlm-grc-en'
#     print(f"\nLoading model: {model_name}")
#     model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
#     tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
    
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     print(f"Using device: {device}")
#     model = model.to(device)
#     model.eval()
    
#     # Compute embeddings
#     print("\nComputing embeddings...")
#     verses = df['verse'].tolist()
#     embeddings = []
#     batch_size = 32
    
#     for i in tqdm(range(0, len(verses), batch_size), desc="Computing embeddings"):
#         try:
#             batch = verses[i:i+batch_size]
#             encoded_input = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')
#             encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
            
#             with torch.no_grad():
#                 model_output = model(**encoded_input)
            
#             batch_embeddings = cls_pooling(model_output).cpu().numpy()
#             embeddings.append(batch_embeddings)
#         except Exception as e:
#             print(f"\n⚠️ Error at batch {i}: {e}")
#             embeddings.append(np.zeros((len(batch), model.config.hidden_size), dtype=np.float32))
    
#     embeddings = np.vstack(embeddings)
#     embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
    
#     # Create dataset mapping
#     source_datasets = df['source_dataset'].values
#     dataset_to_indices = defaultdict(list)
#     for idx, dataset in enumerate(source_datasets):
#         dataset_to_indices[dataset].append(idx)

# else:
#     print("⏭️  Skipping data loading and embedding computation (loaded from checkpoint)")


# # ============================================================================
# # NOTEBOOK CELL 4: FAISS IVF + SAVE COMPACT CHECKPOINT (only if no checkpoint)
# # ============================================================================
# if not LOADED_FROM_CHECKPOINT:
#     print("\n" + "="*80)
#     print("STAGE 1: FAISS IVF PREFILTERING")
#     print("="*80)
    
#     try:
#         import faiss
        
#         dimension = embeddings.shape[1]
#         nlist = 500
#         k = 100
#         nprobe = 20
        
#         print(f"\nFAISS: dimension={dimension}, nlist={nlist}, k={k}, nprobe={nprobe}")
        
#         quantizer = faiss.IndexFlatIP(dimension)
#         index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_INNER_PRODUCT)
        
#         print("Training...")
#         index.train(embeddings.astype('float32'))
        
#         print("Adding vectors...")
#         index.add(embeddings.astype('float32'))
        
#         print("Searching...")
#         index.nprobe = nprobe
#         distances, indices = index.search(embeddings.astype('float32'), k)
        
#         print("Filtering for cross-dataset pairs...")
#         candidate_pairs = set()
#         similarity_threshold = 0.75
        
#         for i in tqdm(range(len(embeddings)), desc="Building candidate graph"):
#             dataset_i = source_datasets[i]
#             for j_idx, distance in zip(indices[i], distances[i]):
#                 if j_idx == i or j_idx == -1:
#                     continue
#                 dataset_j = source_datasets[j_idx]
#                 if dataset_i != dataset_j and distance >= similarity_threshold:
#                     idx1, idx2 = min(i, j_idx), max(i, j_idx)
#                     candidate_pairs.add((idx1, idx2))
        
#         print(f"Cross-dataset candidate pairs: {len(candidate_pairs):,}")
#         faiss_available = True
        
#     except ImportError:
#         print("❌ FAISS not installed!")
#         faiss_available = False
#         candidate_pairs = set()

#     if faiss_available and len(candidate_pairs) > 0:
#         print("\n" + "="*80)
#         print("💾 SAVING COMPACT CHECKPOINT")
#         print("="*80)
        
#         # Embeddings (compressed numpy)
#         np.savez_compressed(CHECKPOINT_DIR / 'embeddings.npz', embeddings=embeddings)
        
#         # Candidate pairs (compressed numpy)
#         pairs_array = np.array(list(candidate_pairs), dtype=np.int32)
#         np.savez_compressed(CHECKPOINT_DIR / 'candidate_pairs.npz', pairs=pairs_array)
        
#         # DataFrame (parquet with compression)
#         essential_cols = ['verse', 'source_dataset']
#         for col in ['idoriginal_poem', 'idgroup']:
#             if col in df.columns:
#                 essential_cols.append(col)
        
#         df_minimal = df[essential_cols].copy()
        
#         # Fix data types for parquet compatibility
#         for col in df_minimal.columns:
#             if df_minimal[col].dtype == 'object':
#                 df_minimal[col] = df_minimal[col].astype(str)
#                 df_minimal[col] = df_minimal[col].replace('nan', None).replace('None', None)
        
#         df_minimal.to_parquet(CHECKPOINT_DIR / 'df_minimal.parquet', 
#                               compression='gzip', index=True)
        
#         # Metadata (gzip pickle)
#         metadata = {
#             'source_datasets': source_datasets,
#             'dataset_to_indices': dataset_to_indices,
#             'similarity_threshold': similarity_threshold,
#             'faiss_config': {'dimension': dimension, 'nlist': nlist, 'k': k, 'nprobe': nprobe}
#         }
#         with gzip.open(CHECKPOINT_DIR / 'metadata.pkl.gz', 'wb') as f:
#             pickle.dump(metadata, f, protocol=pickle.HIGHEST_PROTOCOL)
        
#         # Calculate size
#         total_size = sum(os.path.getsize(f) / 1024 / 1024 
#                         for f in CHECKPOINT_DIR.glob('*') 
#                         if f.is_file() and f.suffix in ['.npz', '.parquet', '.gz'])
        
#         print(f"\n✅ Compact checkpoint saved: {total_size:.1f} MB")
#         print("\n" + "="*80)
#         print("YOU CAN NOW STOP THE NOTEBOOK")
#         print("="*80)

# else:
#     print("⏭️  Skipping FAISS stage (loaded from checkpoint)")


# # ============================================================================
# # NOTEBOOK CELL 6: SAVE FINAL RESULTS
# # ============================================================================
# print("\n" + "="*80)
# print("SAVING RESULTS")
# print("="*80)

# df.to_csv("concatenated_cross_dataset_clusters.csv", index=False)
# print("✓ Full results saved")

# # Save cross-dataset verses only
# cross_mask = df['cluster_id'].map(lambda cid: len(cluster_datasets[cid]) > 1)
# df_cross = df[cross_mask].copy()
# df_cross.to_csv("cross_dataset_verses_only.csv", index=False)
# print(f"✓ Cross-dataset verses: {len(df_cross):,}")

# print("\n" + "="*80)
# print("✅ COMPLETE")
# print("="*80)
import re
import unicodedata
from typing import Dict
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel
import torch
from tqdm import tqdm
from collections import defaultdict
import time
import os
import pickle
import gzip
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import igraph as ig
import leidenalg as la

CHECKPOINT_DIR = Path("/scratch/gent/vo/000/gvo00042/vsc48660/full_semantic_clustering_checkpoints")
RESULTS_DIR = Path("semantic_clustering_results")
CHECKPOINT_DIR.mkdir(exist_ok=True)
RESULTS_DIR.mkdir(exist_ok=True)

print("✓ Imports loaded")
print(f"✓ Checkpoint directory: {CHECKPOINT_DIR}")
print(f"✓ Results directory: {RESULTS_DIR}")

CLEAN_PATTERN = re.compile(r'[^\w\s]')
WHITESPACE_PATTERN = re.compile(r'\s+')

def preprocess_text(text: str, options: Dict[str, bool] = None) -> str:
    if options is None:
        options = {'lowercase': True, 'remove_diacritics': True, 'remove_punctuation': True}
    text = str(text)
    if options.get('lowercase', True):
        text = text.lower()
    if options.get('remove_diacritics', True):
        text = unicodedata.normalize('NFD', text)
        text = ''.join(char for char in text if unicodedata.category(char) != 'Mn')
        text = unicodedata.normalize('NFC', text)
    else:
        text = unicodedata.normalize('NFC', text)
    if options.get('remove_punctuation', True):
        text = CLEAN_PATTERN.sub('', text)
    text = WHITESPACE_PATTERN.sub(' ', text)
    return text.strip()

def cls_pooling(model_output):
    return model_output[0][:, 0]

def stratified_sample(df, n_sample=10000):
    datasets = df['source_dataset'].unique()
    total_size = len(df)
    sample_indices = []
    
    for dataset in datasets:
        dataset_indices = df[df['source_dataset'] == dataset].index.tolist()
        dataset_size = len(dataset_indices)
        proportion = dataset_size / total_size
        n_from_dataset = int(n_sample * proportion)
        n_from_dataset = min(n_from_dataset, dataset_size)
        sampled = np.random.choice(dataset_indices, size=n_from_dataset, replace=False)
        sample_indices.extend(sampled)
    
    return sorted(sample_indices)

print("✓ Utility functions defined")

embeddings_file = CHECKPOINT_DIR / 'embeddings.npz'
metadata_file = CHECKPOINT_DIR / 'metadata.pkl.gz'

if embeddings_file.exists() and metadata_file.exists():
    print("="*80)
    print("📂 COMPACT CHECKPOINT FOUND - LOADING FROM DISK")
    print("="*80)
    
    print("\nLoading embeddings...")
    embeddings_data = np.load(embeddings_file)
    embeddings = embeddings_data['embeddings']
    print(f"  ✓ Embeddings: {embeddings.shape}")
    
    pairs_file = CHECKPOINT_DIR / 'candidate_pairs.npz'
    if pairs_file.exists():
        print("Loading candidate pairs...")
        pairs_data = np.load(pairs_file)
        pairs_array = pairs_data['pairs']
        candidate_pairs = set((int(i), int(j)) for i, j in pairs_array)
        print(f"  ✓ Candidate pairs: {len(candidate_pairs):,}")
    else:
        candidate_pairs = set()
    
    print("Loading DataFrame...")
    df = pd.read_parquet(CHECKPOINT_DIR / 'df_minimal.parquet')
    print(f"  ✓ DataFrame: {len(df):,} rows")
    
    print("Loading metadata...")
    with gzip.open(metadata_file, 'rb') as f:
        metadata = pickle.load(f)
    
    source_datasets = metadata['source_datasets']
    dataset_to_indices = metadata['dataset_to_indices']
    similarity_threshold = metadata['similarity_threshold']
    faiss_config = metadata['faiss_config']
    
    dimension = faiss_config['dimension']
    nlist = faiss_config['nlist']
    k = faiss_config['k']
    nprobe = faiss_config['nprobe']
    
    total_size = sum(os.path.getsize(f) / 1024 / 1024 
                     for f in CHECKPOINT_DIR.glob('*') 
                     if f.is_file() and f.suffix in ['.npz', '.parquet', '.gz'])
    
    print(f"\n✅ Checkpoint loaded! Size: {total_size:.1f} MB")
    print(f"  Documents: {len(df):,}")
    print(f"  Candidate pairs: {len(candidate_pairs):,}")
    print(f"  Source datasets: {list(dataset_to_indices.keys())}")
    
    print("\n" + "="*80)
    print("✅ READY TO PROCEED WITH THRESHOLD/RESOLUTION SWEEP")
    print("="*80)
    
    LOADED_FROM_CHECKPOINT = True
    faiss_available = True
    
else:
    print("="*80)
    print("ℹ️  NO CHECKPOINT FOUND")
    print("="*80)
    print("Will run pipeline from the beginning.\n")
    LOADED_FROM_CHECKPOINT = False

if not LOADED_FROM_CHECKPOINT:
    print("="*80)
    print("LOADING DATA")
    print("="*80)
    
    csv_path = 'concatenated.csv'
    df = pd.read_csv(csv_path)
    df = df.dropna(subset=['verse', 'source_dataset'])
    df = df[df['verse'].fillna('').astype(str).str.len() >= 20]
    df = df[df['verse'].fillna('').astype(str).str.len() < 256]
    df['verse'] = df['verse'].apply(preprocess_text)
    df = df.reset_index(drop=True)
    
    print(f"Total verses: {len(df):,}")
    print(f"Dataset distribution:")
    print(df['source_dataset'].value_counts())
    
    model_name = 'kevinkrahn/shlm-grc-en'
    print(f"\nLoading model: {model_name}")
    
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()
    
    try:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {device}")
        
        model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
        tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        model = model.to(device)
        model.eval()
    except RuntimeError as e:
        print(f"CUDA error: {e}")
        print("Falling back to CPU")
        device = torch.device('cpu')
        model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
        tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
        model = model.to(device)
        model.eval()
    
    print("\nComputing embeddings...")
    verses = df['verse'].tolist()
    embeddings = []
    batch_size = 32
    
    for i in tqdm(range(0, len(verses), batch_size), desc="Computing embeddings"):
        try:
            batch = verses[i:i+batch_size]
            encoded_input = tokenizer(batch, padding=True, truncation=True, return_tensors='pt')
            encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
            
            with torch.no_grad():
                model_output = model(**encoded_input)
            
            batch_embeddings = cls_pooling(model_output).cpu().numpy()
            embeddings.append(batch_embeddings)
        except Exception as e:
            print(f"\n⚠️ Error at batch {i}: {e}")
            embeddings.append(np.zeros((len(batch), model.config.hidden_size), dtype=np.float32))
    
    embeddings = np.vstack(embeddings)
    embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
    
    source_datasets = df['source_dataset'].values
    dataset_to_indices = defaultdict(list)
    for idx, dataset in enumerate(source_datasets):
        dataset_to_indices[dataset].append(idx)

else:
    print("⏭️  Skipping data loading and embedding computation (loaded from checkpoint)")

if not LOADED_FROM_CHECKPOINT:
    print("\n" + "="*80)
    print("STAGE 1: FAISS IVF PREFILTERING")
    print("="*80)
    
    try:
        import faiss
        
        dimension = embeddings.shape[1]
        nlist = 500
        k = 100
        nprobe = 20
        
        print(f"\nFAISS: dimension={dimension}, nlist={nlist}, k={k}, nprobe={nprobe}")
        
        quantizer = faiss.IndexFlatIP(dimension)
        index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_INNER_PRODUCT)
        
        print("Training...")
        index.train(embeddings.astype('float32'))
        
        print("Adding vectors...")
        index.add(embeddings.astype('float32'))
        
        print("Searching...")
        index.nprobe = nprobe
        distances, indices = index.search(embeddings.astype('float32'), k)
        
        print("Filtering for cross-dataset pairs...")
        candidate_pairs = set()
        similarity_threshold = 0.75
        
        for i in tqdm(range(len(embeddings)), desc="Building candidate graph"):
            dataset_i = source_datasets[i]
            for j_idx, distance in zip(indices[i], distances[i]):
                if j_idx == i or j_idx == -1:
                    continue
                dataset_j = source_datasets[j_idx]
                if dataset_i != dataset_j and distance >= similarity_threshold:
                    idx1, idx2 = min(i, j_idx), max(i, j_idx)
                    candidate_pairs.add((idx1, idx2))
        
        print(f"Cross-dataset candidate pairs: {len(candidate_pairs):,}")
        faiss_available = True
        
    except ImportError:
        print("❌ FAISS not installed!")
        faiss_available = False
        candidate_pairs = set()

    if faiss_available and len(candidate_pairs) > 0:
        print("\n" + "="*80)
        print("💾 SAVING COMPACT CHECKPOINT")
        print("="*80)
        
        np.savez_compressed(CHECKPOINT_DIR / 'embeddings.npz', embeddings=embeddings)
        
        pairs_array = np.array(list(candidate_pairs), dtype=np.int32)
        np.savez_compressed(CHECKPOINT_DIR / 'candidate_pairs.npz', pairs=pairs_array)
        
        essential_cols = ['verse', 'source_dataset']
        for col in ['idoriginal_poem', 'idgroup']:
            if col in df.columns:
                essential_cols.append(col)
        
        df_minimal = df[essential_cols].copy()
        
        for col in df_minimal.columns:
            if df_minimal[col].dtype == 'object':
                df_minimal[col] = df_minimal[col].astype(str)
                df_minimal[col] = df_minimal[col].replace('nan', None).replace('None', None)
        
        df_minimal.to_parquet(CHECKPOINT_DIR / 'df_minimal.parquet', 
                              compression='gzip', index=True)
        
        metadata = {
            'source_datasets': source_datasets,
            'dataset_to_indices': dataset_to_indices,
            'similarity_threshold': similarity_threshold,
            'faiss_config': {'dimension': dimension, 'nlist': nlist, 'k': k, 'nprobe': nprobe}
        }
        with gzip.open(CHECKPOINT_DIR / 'metadata.pkl.gz', 'wb') as f:
            pickle.dump(metadata, f, protocol=pickle.HIGHEST_PROTOCOL)
        
        total_size = sum(os.path.getsize(f) / 1024 / 1024 
                        for f in CHECKPOINT_DIR.glob('*') 
                        if f.is_file() and f.suffix in ['.npz', '.parquet', '.gz'])
        
        print(f"\n✅ Compact checkpoint saved: {total_size:.1f} MB")

else:
    print("⏭️  Skipping FAISS stage (loaded from checkpoint)")

print("\n" + "="*80)
print("STAGE 2: THRESHOLD/RESOLUTION SWEEP ON SAMPLE")
print("="*80)

sample_indices = stratified_sample(df, n_sample=10000)
print(f"Sample size: {len(sample_indices):,}")

sample_pairs = set()
for i, j in candidate_pairs:
    if i in sample_indices and j in sample_indices:
        sample_pairs.add((i, j))

print(f"Sample pairs: {len(sample_pairs):,}")

similarity_thresholds = [0.80, 0.85, 0.90]
all_results = []

for sim_threshold in similarity_thresholds:
    print(f"\n{'='*80}")
    print(f"Testing Threshold: {sim_threshold:.2f}")
    print(f"{'='*80}")
    
    threshold_pairs = set()
    for i, j in tqdm(sample_pairs, desc="Filtering by threshold"):
        similarity = float(np.dot(embeddings[i], embeddings[j]))
        if similarity >= sim_threshold:
            threshold_pairs.add((i, j))
    
    n_pairs = len(threshold_pairs)
    avg_degree = n_pairs * 2 / len(sample_indices) if len(sample_indices) > 0 else 0
    
    print(f"Pairs at threshold: {n_pairs:,} (avg degree: {avg_degree:.1f})")
    
    if n_pairs == 0:
        print("No pairs found - skipping")
        continue
    
    edges = [(i, j) for i, j in threshold_pairs]
    weights = [float(np.dot(embeddings[i], embeddings[j])) for i, j in edges]
    
    g = ig.Graph(n=len(df), edges=edges, directed=False)
    g.es['weight'] = weights
    
    print(f"Graph: {g.vcount()} nodes, {g.ecount()} edges")
    
    w = np.array(weights)
    w_scaled = ((w - w.min()) / (w.max() - w.min())) ** 3
    g.es['weight'] = w_scaled.tolist()
    
    hub_thresh = 500
    for v in range(g.vcount()):
        if g.degree(v) > hub_thresh:
            for e in g.incident(v):
                g.es[e]['weight'] *= 0.5
    
    print("Leiden resolution sweep...")
    resolutions = np.logspace(-2, 1, 15)
    
    for res in tqdm(resolutions, desc="Resolutions"):
        partition = la.find_partition(
            g,
            la.CPMVertexPartition,
            weights='weight',
            resolution_parameter=res,
            n_iterations=-1
        )
        labels = np.array(partition.membership)
        
        cluster_datasets = defaultdict(set)
        for idx in sample_indices:
            cluster_id = labels[idx]
            cluster_datasets[cluster_id].add(source_datasets[idx])
        
        n_cross_dataset = sum(1 for datasets in cluster_datasets.values() if len(datasets) > 1)
        n_clusters = len(set(labels))
        cluster_sizes = np.bincount(labels)
        avg_cluster_size = cluster_sizes.mean()
        
        all_results.append({
            'threshold': sim_threshold,
            'resolution': res,
            'n_clusters': n_clusters,
            'n_cross_dataset_clusters': n_cross_dataset,
            'avg_cluster_size': avg_cluster_size,
            'n_pairs': n_pairs
        })
        
        print(f"  res={res:.4f}: clusters={n_clusters}, cross-dataset={n_cross_dataset}")

results_df = pd.DataFrame(all_results)
results_df.to_csv(RESULTS_DIR / "threshold_resolution_sweep.csv", index=False)

print("\n" + "="*80)
print("THRESHOLD/RESOLUTION SWEEP RESULTS")
print("="*80)
print(results_df.to_string(index=False))

pivot_clusters = results_df.pivot(index='resolution', columns='threshold', values='n_cross_dataset_clusters')

plt.figure(figsize=(10, 8))
sns.heatmap(pivot_clusters, annot=True, fmt='.0f', cmap='viridis', 
            cbar_kws={'label': 'Cross-dataset Clusters'},
            annot_kws={"fontsize":12})
plt.ylabel("Resolution", fontsize=12)
plt.xlabel("Similarity Threshold", fontsize=12)
plt.title("Cross-dataset Clusters by Threshold and Resolution", fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(RESULTS_DIR / "threshold_resolution_heatmap.png", dpi=300)
plt.close()

best_row = results_df[results_df['threshold'] >= 0.85].sort_values('n_cross_dataset_clusters', ascending=False).iloc[0]
best_threshold = best_row['threshold']
best_resolution = best_row['resolution']

print(f"\n{'='*80}")
print(f"BEST CONFIGURATION (threshold >= 0.85):")
print(f"{'='*80}")
print(f"Threshold: {best_threshold:.2f}")
print(f"Resolution: {best_resolution:.4f}")
print(f"Cross-dataset clusters: {int(best_row['n_cross_dataset_clusters'])}")
print(f"Total clusters: {int(best_row['n_clusters'])}")

print("\n" + "="*80)
print("STAGE 3: FULL CLUSTERING WITH BEST PARAMETERS")
print("="*80)

threshold_pairs = set()
for i, j in tqdm(candidate_pairs, desc="Filtering by best threshold"):
    similarity = float(np.dot(embeddings[i], embeddings[j]))
    if similarity >= best_threshold:
        threshold_pairs.add((i, j))

print(f"Pairs at threshold {best_threshold:.2f}: {len(threshold_pairs):,}")

edges = [(i, j) for i, j in threshold_pairs]
weights = [float(np.dot(embeddings[i], embeddings[j])) for i, j in edges]

g = ig.Graph(n=len(df), edges=edges, directed=False)
g.es['weight'] = weights

w = np.array(weights)
w_scaled = ((w - w.min()) / (w.max() - w.min())) ** 3
g.es['weight'] = w_scaled.tolist()

hub_thresh = 500
for v in range(g.vcount()):
    if g.degree(v) > hub_thresh:
        for e in g.incident(v):
            g.es[e]['weight'] *= 0.5

print(f"Running Leiden with resolution={best_resolution:.4f}...")
partition = la.find_partition(
    g,
    la.CPMVertexPartition,
    weights='weight',
    resolution_parameter=best_resolution,
    n_iterations=-1
)

cluster_labels = np.array(partition.membership)
df['cluster_id'] = cluster_labels

cluster_datasets = defaultdict(set)
for idx, cluster_id in enumerate(cluster_labels):
    cluster_datasets[cluster_id].add(source_datasets[idx])

n_cross_dataset = sum(1 for datasets in cluster_datasets.values() if len(datasets) > 1)
print(f"Cross-dataset clusters: {n_cross_dataset:,}")

print("\n" + "="*80)
print("SAVING RESULTS")
print("="*80)

df.to_csv(RESULTS_DIR / "concatenated_cross_dataset_clusters.csv", index=False)
print("✓ Full results saved")

cross_mask = df['cluster_id'].map(lambda cid: len(cluster_datasets[cid]) > 1)
df_cross = df[cross_mask].copy()
df_cross.to_csv(RESULTS_DIR / "cross_dataset_verses_only.csv", index=False)
print(f"✓ Cross-dataset verses: {len(df_cross):,}")

print("\n" + "="*80)
print("✅ COMPLETE")
print("="*80)

✓ Imports loaded
✓ Checkpoint directory: /scratch/gent/vo/000/gvo00042/vsc48660/full_semantic_clustering_checkpoints
✓ Results directory: semantic_clustering_results
✓ Utility functions defined
📂 COMPACT CHECKPOINT FOUND - LOADING FROM DISK

Loading embeddings...
  ✓ Embeddings: (1536616, 768)
Loading candidate pairs...
  ✓ Candidate pairs: 35,082,746
Loading DataFrame...
  ✓ DataFrame: 1,536,616 rows
Loading metadata...

✅ Checkpoint loaded! Size: 4358.6 MB
  Documents: 1,536,616
  Candidate pairs: 35,082,746
  Source datasets: ['rhoby', 'dbbe', 'phi', 'papyri']

✅ READY TO PROCEED WITH THRESHOLD/RESOLUTION SWEEP
⏭️  Skipping data loading and embedding computation (loaded from checkpoint)
⏭️  Skipping FAISS stage (loaded from checkpoint)

STAGE 2: THRESHOLD/RESOLUTION SWEEP ON SAMPLE
Sample size: 9,998
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/apps/gent/RHEL9/zen3-ampere-ib/software/IPython/8.14.0-GCCcore-12.3.0/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_305272/1553198463.py", line -1, in <module>
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/apps/gent/RHEL9/zen3-ampere-ib/software/IPython/8.14.0-GCCcore-12.3.0/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2105, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/apps/gent/RHEL9/zen3-ampere-ib/software/IPython/8.14.0-GCCcore-12.3.0/lib/python3.11/site-packages/IPython/core/ultratb.py", line 1428, in structured_traceback
    return FormattedTB.structured_traceback(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/apps/gent/RHEL

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
from typing import List, Dict, Set, Tuple
from sklearn.metrics import adjusted_rand_score, v_measure_score
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import pandas as pd

def calculate_jaccard_similarity(clusters_a: Set[int], clusters_b: Set[int]) -> float:
    if not clusters_a or not clusters_b:
        return 0.0
    intersection = len(clusters_a & clusters_b)
    union = len(clusters_a | clusters_b)
    return intersection / union

def reconstruct_poems_unsupervised(df, cluster_col='cluster_id'):
    poem_to_clusters = defaultdict(set)
    poem_verse_counts = defaultdict(int)

    for _, row in df.iterrows():
        poem_id = row['idoriginal_poem']
        cluster_id = row[cluster_col]
        poem_verse_counts[poem_id] += 1
        if cluster_id != -1:
            poem_to_clusters[poem_id].add(cluster_id)

    print(f"Reconstructed {len(poem_to_clusters)} poems")
    return poem_to_clusters, poem_verse_counts

def evaluate_against_ground_truth(df, poem_clusters):
    poem_to_type = df.groupby('idoriginal_poem')['type_id'].first().to_dict()

    y_true = []
    y_pred = []
    for poem_id, predicted_cluster in poem_clusters.items():
        if poem_id in poem_to_type:
            y_true.append(poem_to_type[poem_id])
            y_pred.append(predicted_cluster)

    ari = adjusted_rand_score(y_true, y_pred)
    v_measure = v_measure_score(y_true, y_pred)

    return ari, v_measure, y_true, y_pred

def cluster_poems_jaccard_unsupervised(poem_to_clusters: dict, similarity_threshold: float = 0.66):
    poem_ids = list(poem_to_clusters.keys())
    n_poems = len(poem_ids)
    
    edges = []

    for i in tqdm(range(n_poems), desc="Comparing poems"):
        for j in range(i+1, n_poems):
            pid_a, pid_b = poem_ids[i], poem_ids[j]
            set_a, set_b = poem_to_clusters[pid_a], poem_to_clusters[pid_b]
            if not set_a or not set_b:
                continue
            sim = len(set_a & set_b) / len(set_a | set_b)
            if sim >= similarity_threshold:
                edges.append((pid_a, pid_b))
    
    class UF:
        def __init__(self, elements):
            self.parent = {e: e for e in elements}
            self.rank = {e: 0 for e in elements}
        def find(self, x):
            if self.parent[x] != x:
                self.parent[x] = self.find(self.parent[x])
            return self.parent[x]
        def union(self, x, y):
            px, py = self.find(x), self.find(y)
            if px == py: return
            if self.rank[px] < self.rank[py]: px, py = py, px
            self.parent[py] = px
            if self.rank[px] == self.rank[py]: self.rank[px] += 1
    
    uf = UF(poem_ids)
    for a, b in edges:
        uf.union(a, b)
    
    poem_clusters = {pid: uf.find(pid) for pid in poem_ids}
    return poem_clusters, edges


def calculate_perfect_reconstruction_rate(df, poem_clusters):
    poem_to_type = df.groupby('idoriginal_poem')['type_id'].first().to_dict()

    gt_to_poems = defaultdict(set)
    for poem_id, gt_type in poem_to_type.items():
        gt_to_poems[gt_type].add(poem_id)

    pred_to_poems = defaultdict(set)
    for poem_id, pred_cluster in poem_clusters.items():
        pred_to_poems[pred_cluster].add(poem_id)

    perfectly_reconstructed = 0
    total_gt_clusters = len(gt_to_poems)

    for gt_type, gt_poems in gt_to_poems.items():
        for pred_cluster, pred_poems in pred_to_poems.items():
            if gt_poems == pred_poems:
                perfectly_reconstructed += 1
                break

    reconstruction_rate = perfectly_reconstructed / total_gt_clusters if total_gt_clusters > 0 else 0
    return reconstruction_rate, perfectly_reconstructed, total_gt_clusters


poem_to_clusters, _ = reconstruct_poems_unsupervised(df)

thresholds = [0.50, 0.60, 0.66, 0.70]
results = []

for thresh in thresholds:
    print(f"\nThreshold {thresh:.0%}...")
    poem_clusters, edges = cluster_poems_jaccard_unsupervised(poem_to_clusters, thresh)
    df['poem_cluster_id'] = df['idoriginal_poem'].map(poem_clusters)

    results.append({
        'threshold': thresh,
        'n_poem_clusters': len(set(poem_clusters.values())),
        'n_edges': len(edges),
    })
    display(results)

results_df = pd.DataFrame(results)


In [81]:
pd.read_csv('"concatenated_cross_dataset_clusters.csv")

SyntaxError: unterminated string literal (detected at line 1) (161000975.py, line 1)